<a href="https://colab.research.google.com/github/Gopalprajaapati/DomainSocialMediaCollector/blob/main/code_social_medial_with_domain_whois_and_website_value_estimated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-whois

import whois

# Specify the domain name
domain = whois.whois("https://www.thesocialco.co.uk")

# Print WHOIS information
print("Domain Name:", domain.domain_name)
print("Registrar:", domain.registrar)
print("Creation Date:", domain.creation_date)
print("Expiration Date:", domain.expiration_date)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 4.2 MB/s eta 0:00:00
Domain Name: thesocialco.co.uk
Registrar: ANS Group Ltd [Tag = UKFAST]
Creation Date: 2017-08-16 00:00:00
Expiration Date: 2026-08-16 00:00:00


In [ ]:
!pip install python-whois pandas beautifulsoup4 openpyxl requests


In [ ]:
!pip install python-whois pandas beautifulsoup4 openpyxl requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import whois

# Function to extract emails and phone numbers from a given URL
def extract_emails_and_phones(url):
    try:
        web = requests.get(url)
        soup = BeautifulSoup(web.content, "html.parser")

        # Updated email regex to cleanly extract only the email
        email_pattern = r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'
        emails = re.findall(email_pattern, soup.get_text())
        cleaned_emails = [re.sub(r'[^a-zA-Z0-9._%+-@]', '', email) for email in emails]

        # Updated phone pattern to capture complete numbers
        phone_pattern = r'(\+?\d{1,4}[\s-]?)?(\(?\d{2,4}\)?[\s-]?){0,1}(\d{3,4}[\s-]?\d{3,4}[\s-]?\d{0,4})'
        phone_numbers = [''.join(number).strip() for number in re.findall(phone_pattern, soup.get_text())]
        phone_numbers = list(set(phone_numbers))

        return cleaned_emails, phone_numbers, soup
    except Exception as e:
        print(f"Error fetching data from {url}: {e}")
        return [], [], None

# Function to check for WordPress and its version
def check_wordpress(soup, url):
    is_wordpress = False
    wordpress_version = None

    if '/wp-content/' in str(soup) or '/wp-admin/' in str(soup):
        is_wordpress = True
        version_meta = soup.find('meta', {'name': 'generator'})
        if version_meta and 'WordPress' in version_meta.get('content', ''):
            match = re.search(r'WordPress (\d+\.\d+\.\d+)', version_meta.get('content'))
            if match:
                wordpress_version = match.group(1)
    return is_wordpress, wordpress_version

# Function to get website language
def get_website_language(soup):
    html_tag = soup.find('html')
    return html_tag.get('lang', 'Unknown') if html_tag else 'Unknown'

# Function to fetch WHOIS data
def fetch_whois_data(domain_url):
    try:
        domain_info = whois.whois(domain_url)
        return {
            'domain_name': domain_info.domain_name,
            'registrar': domain_info.registrar,
            'creation_date': domain_info.creation_date,
            'expiration_date': domain_info.expiration_date
        }
    except Exception as e:
        print(f"WHOIS lookup failed for {domain_url}: {e}")
        return {
            'domain_name': 'Not available',
            'registrar': 'Not available',
            'creation_date': 'Not available',
            'expiration_date': 'Not available'
        }

# Load URLs from the Excel file
excel_file_path = 'url.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file_path)
urls = df['URL'].dropna().tolist()

# List to store the results
results = []

# Iterate over each URL
for main_url in urls:
    print(f"Processing URL: {main_url}")

    try:
        web = requests.get(main_url)
        soup = BeautifulSoup(web.content, "html.parser")

        # Extract social media links
        all_links = [a['href'] for a in soup.find_all('a', href=True)]
        linkedin = next((link for link in all_links if "linkedin.com" in link), "Not found")
        instagram = next((link for link in all_links if "instagram.com" in link), "Not found")
        facebook = next((link for link in all_links if "facebook.com" in link), "Not found")
        twitter = next((link for link in all_links if "twitter.com" in link), "Not found")

        # Extract emails and phone numbers
        emails, phone_numbers, soup = extract_emails_and_phones(main_url)

        # Check WordPress and version
        is_wordpress, wordpress_version = check_wordpress(soup, main_url)

        # Get website language
        website_language = get_website_language(soup)

        # Fetch WHOIS data
        domain_info = fetch_whois_data(main_url)

        # If no emails or phones, check common contact pages
        if not emails or not phone_numbers:
            for contact_url in [f"{main_url.rstrip('/')}/contact", f"{main_url.rstrip('/')}/contact-us"]:
                print(f"Checking contact page: {contact_url}...")
                contact_emails, contact_phones, _ = extract_emails_and_phones(contact_url)
                emails = emails or contact_emails
                phone_numbers = phone_numbers or contact_phones

        # Add results to the list
        for email in emails or ["Not found"]:
            results.append({
                'website': main_url,
                'linkedin': linkedin,
                'instagram': instagram,
                'facebook': facebook,
                'twitter': twitter,
                'email': email,
                'number': ', '.join(phone_numbers) if phone_numbers else "Not found",
                'language': website_language,
                'is_wordpress': 'Yes' if is_wordpress else 'No',
                'wordpress_version': wordpress_version or 'Not available',
                'domain_name': domain_info['domain_name'],
                'registrar': domain_info['registrar'],
                'creation_date': domain_info['creation_date'],
                'expiration_date': domain_info['expiration_date']
            })

    except Exception as e:
        print(f"Error processing {main_url}: {e}")
        results.append({
            'website': main_url,
            'linkedin': "Error processing",
            'instagram': "Error processing",
            'facebook': "Error processing",
            'twitter': "Error processing",
            'email': "Error processing",
            'number': "Error processing",
            'language': "Error processing",
            'is_wordpress': "Error processing",
            'wordpress_version': "Error processing",
            'domain_name': "Error processing",
            'registrar': "Error processing",
            'creation_date': "Error processing",
            'expiration_date': "Error processing"
        })

# Convert results into a DataFrame
results_df = pd.DataFrame(results)
print("\nResults Table:")
print(results_df)

# Save the results to a new Excel file
output_excel_path = 'scraped_data_with_whois.xlsx'
results_df.to_excel(output_excel_path, index=False)
print(f"\nResults saved to {output_excel_path}")


ERROR: Could not find a version that satisfies the requirement requestsimport (from versions: none)
ERROR: No matching distribution found for requestsimport


ModuleNotFoundError: No module named 'whois'

In [ ]:
!pip install python-whois pandas beautifulsoup4 openpyxl requests
import whois
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urlparse


# 1. Domain Age ko calculate karna
def get_domain_age(domain_name):
    try:
        domain_info = whois.whois(domain_name)
        creation_date = domain_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        age = (datetime.now() - creation_date).days // 365  # Age in years
        return age
    except Exception as e:
        print(f"Error fetching domain age: {e}")
        return 0

# 2. Backlinks ko scrape karna
def get_unique_backlinks(domain_name):
    try:
        # Parse the domain name from the full URL if needed
        parsed_url = urlparse(domain_name)
        # Use only the domain (hostname) part of the URL
        domain = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        # Make sure we're using the full "http" or "https" scheme
        url = f"http://{domain}" if not parsed_url.scheme else domain_name

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        unique_backlinks = set([link.get('href') for link in links if link.get('href')])
        return len(unique_backlinks)
    except Exception as e:
        print(f"Error fetching backlinks for {domain_name}: {e}")
        return 0

# 3. Website Value ko estimate karna
def estimate_value(domain_name, monthly_revenue):
    # Domain Age
    domain_age = get_domain_age(domain_name)

    # Backlinks count
    backlinks_count = get_unique_backlinks(domain_name)

    # Simple valuation formula
    estimated_value = monthly_revenue * 24 + (domain_age * 5000) + (backlinks_count * 100)

    return estimated_value

# Example usage
domain_name = "https://driftwoodmediapro.co.uk"  # Apne domain ka naam yahan daalein
monthly_revenue = 5000  # Monthly revenue ka estimate

value = estimate_value(domain_name, monthly_revenue)

print(f"Estimated Website Value for {domain_name}: ₹{value}")


Estimated Website Value for https://driftwoodmediapro.co.uk: ₹152800


In [ ]:
!pip install python-whois pandas beautifulsoup4 openpyxl requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import whois
from datetime import datetime
from urllib.parse import urlparse


# Function to extract emails and phone numbers from a given URL
def extract_emails_and_phones(url):
    try:
        web = requests.get(url)
        soup = BeautifulSoup(web.content, "html.parser")

        # Updated email regex to cleanly extract only the email
        email_pattern = r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'
        emails = re.findall(email_pattern, soup.get_text())
        cleaned_emails = [re.sub(r'[^a-zA-Z0-9._%+-@]', '', email) for email in emails]

        # Updated phone pattern to capture complete numbers
        phone_pattern = r'(\+?\d{1,4}[\s-]?)?(\(?\d{2,4}\)?[\s-]?){0,1}(\d{3,4}[\s-]?\d{3,4}[\s-]?\d{0,4})'
        phone_numbers = [''.join(number).strip() for number in re.findall(phone_pattern, soup.get_text())]
        phone_numbers = list(set(phone_numbers))

        return cleaned_emails, phone_numbers, soup
    except Exception as e:
        print(f"Error fetching data from {url}: {e}")
        return [], [], None

# Function to check for WordPress and its version
def check_wordpress(soup, url):
    is_wordpress = False
    wordpress_version = None

    if '/wp-content/' in str(soup) or '/wp-admin/' in str(soup):
        is_wordpress = True
        version_meta = soup.find('meta', {'name': 'generator'})
        if version_meta and 'WordPress' in version_meta.get('content', ''):
            match = re.search(r'WordPress (\d+\.\d+\.\d+)', version_meta.get('content'))
            if match:
                wordpress_version = match.group(1)
    return is_wordpress, wordpress_version

# Function to get website language
def get_website_language(soup):
    html_tag = soup.find('html')
    return html_tag.get('lang', 'Unknown') if html_tag else 'Unknown'

# 1. Domain Age ko calculate karna
def get_domain_age(domain_name):
    try:
        domain_info = whois.whois(domain_name)
        creation_date = domain_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        age = (datetime.now() - creation_date).days // 365  # Age in years
        return age
    except Exception as e:
        print(f"Error fetching domain age: {e}")
        return 0

# 2. Backlinks ko scrape karna
def get_unique_backlinks(domain_name):
    try:
        # Parse the domain name from the full URL if needed
        parsed_url = urlparse(domain_name)
        # Use only the domain (hostname) part of the URL
        domain = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        # Make sure we're using the full "http" or "https" scheme
        url = f"http://{domain}" if not parsed_url.scheme else domain_name

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        unique_backlinks = set([link.get('href') for link in links if link.get('href')])
        return len(unique_backlinks)
    except Exception as e:
        print(f"Error fetching backlinks for {domain_name}: {e}")
        return 0

# 3. Website Value ko estimate karna
def estimate_value(domain_name, monthly_revenue):
    # Domain Age
    domain_age = get_domain_age(domain_name)

    # Backlinks count
    backlinks_count = get_unique_backlinks(domain_name)

    # Simple valuation formula
    estimated_value = monthly_revenue * 24 + (domain_age * 5000) + (backlinks_count * 100)

    return estimated_value

# Function to fetch WHOIS data
def fetch_whois_data(domain_url):
    try:
        domain_info = whois.whois(domain_url)
        return {
            'domain_name': domain_info.domain_name,
            'registrar': domain_info.registrar,
            'creation_date': domain_info.creation_date,
            'expiration_date': domain_info.expiration_date
        }
    except Exception as e:
        print(f"WHOIS lookup failed for {domain_url}: {e}")
        return {
            'domain_name': 'Not available',
            'registrar': 'Not available',
            'creation_date': 'Not available',
            'expiration_date': 'Not available'
        }

# Load URLs from the Excel file
excel_file_path = 'url.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file_path)
urls = df['URL'].dropna().tolist()

# List to store the results
results = []

# Iterate over each URL
for main_url in urls:
    print(f"Processing URL: {main_url}")

    try:
        web = requests.get(main_url)
        soup = BeautifulSoup(web.content, "html.parser")

        # Extract social media links
        all_links = [a['href'] for a in soup.find_all('a', href=True)]
        linkedin = next((link for link in all_links if "linkedin.com" in link), "Not found")
        instagram = next((link for link in all_links if "instagram.com" in link), "Not found")
        facebook = next((link for link in all_links if "facebook.com" in link), "Not found")
        twitter = next((link for link in all_links if "twitter.com" in link), "Not found")

        # Extract emails and phone numbers
        emails, phone_numbers, soup = extract_emails_and_phones(main_url)

        # Check WordPress and version
        is_wordpress, wordpress_version = check_wordpress(soup, main_url)

        # Get website language
        website_language = get_website_language(soup)

        # Fetch WHOIS data
        domain_info = fetch_whois_data(main_url)

        # Estimate website value
        monthly_revenue = 5000  # Example, you can replace it with actual revenue
        estimated_value = estimate_value(main_url, monthly_revenue)

        # If no emails or phones, check common contact pages
        if not emails or not phone_numbers:
            for contact_url in [f"{main_url.rstrip('/')}/contact", f"{main_url.rstrip('/')}/contact-us"]:
                print(f"Checking contact page: {contact_url}...")
                contact_emails, contact_phones, _ = extract_emails_and_phones(contact_url)
                emails = emails or contact_emails
                phone_numbers = phone_numbers or contact_phones

        # Add results to the list
        for email in emails or ["Not found"]:
            results.append({
                'website': main_url,
                'linkedin': linkedin,
                'instagram': instagram,
                'facebook': facebook,
                'twitter': twitter,
                'email': email,
                'number': ', '.join(phone_numbers) if phone_numbers else "Not found",
                'language': website_language,
                'is_wordpress': 'Yes' if is_wordpress else 'No',
                'wordpress_version': wordpress_version or 'Not available',
                'domain_name': domain_info['domain_name'],
                'registrar': domain_info['registrar'],
                'creation_date': domain_info['creation_date'],
                'expiration_date': domain_info['expiration_date'],
                'estimated_value': estimated_value
            })

    except Exception as e:
        print(f"Error processing {main_url}: {e}")
        results.append({
            'website': main_url,
            'linkedin': "Error processing",
            'instagram': "Error processing",
            'facebook': "Error processing",
            'twitter': "Error processing",
            'email': "Error processing",
            'number': "Error processing",
            'language': "Error processing",
            'is_wordpress': "Error processing",
            'wordpress_version': "Error processing",
            'domain_name': "Error processing",
            'registrar': "Error processing",
            'creation_date': "Error processing",
            'expiration_date': "Error processing",
            'estimated_value': "Error processing"
        })

# Convert results into a DataFrame
results_df = pd.DataFrame(results)
print("\nResults Table:")
print(results_df)

# Save the results to a new Excel file
output_excel_path = 'scraped_data_with_whois_and_value.xlsx'
results_df.to_excel(output_excel_path, index=False)
print(f"\nResults saved to {output_excel_path}")


Processing URL: http://climbinfluenceragency.com/
Checking contact page: http://climbinfluenceragency.com/contact...
Checking contact page: http://climbinfluenceragency.com/contact-us...
Processing URL: http://digiful.io/
Processing URL: http://digital-khaos.co.uk/
Checking contact page: http://digital-khaos.co.uk/contact...
Checking contact page: http://digital-khaos.co.uk/contact-us...
Processing URL: http://dirtyjack.co/
Checking contact page: http://dirtyjack.co/contact...
Checking contact page: http://dirtyjack.co/contact-us...
Processing URL: http://ewwatrader.co.uk/
Processing URL: http://ewwatrader.co.uk/
Processing URL: http://digitalfreelance.co.uk/
Processing URL: http://digitalfreelance.co.uk/
Processing URL: http://fearlessmarketing.co.uk/
Checking contact page: http://fearlessmarketing.co.uk/contact...
Checking contact page: http://fearlessmarketing.co.uk/contact-us...
Processing URL: http://concoursemarketing.com/
Processing URL: http://digitalagency.coach/
Checking cont

In [ ]:
!pip install python-whois pandas beautifulsoup4 openpyxl requests
import pandas as pd
import whois
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urlparse

# 1. Domain Age ko calculate karna
def get_domain_age(domain_name):
    try:
        domain_info = whois.whois(domain_name)
        creation_date = domain_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        age = (datetime.now() - creation_date).days // 365  # Age in years
        return age
    except Exception as e:
        print(f"Error fetching domain age: {e}")
        return 0

# 2. Backlinks ko scrape karna
def get_unique_backlinks(domain_name):
    try:
        parsed_url = urlparse(domain_name)
        domain = parsed_url.netloc if parsed_url.netloc else parsed_url.path
        url = f"http://{domain}" if not parsed_url.scheme else domain_name
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        unique_backlinks = set([link.get('href') for link in links if link.get('href')])
        return len(unique_backlinks)
    except Exception as e:
        print(f"Error fetching backlinks for {domain_name}: {e}")
        return 0

# 3. Website Value ko estimate karna
def estimate_value(domain_name, monthly_revenue):
    # Domain Age
    domain_age = get_domain_age(domain_name)

    # Backlinks count
    backlinks_count = get_unique_backlinks(domain_name)

    # Simple valuation formula
    estimated_value = monthly_revenue * 24 + (domain_age * 5000) + (backlinks_count * 100)

    # Calculate monthly and yearly values
    monthly_value = estimated_value
    yearly_value = estimated_value * 12

    return estimated_value, monthly_value, yearly_value

# Example usage
domain_name = "rosechair.com"  # Apne domain ka naam yahan daalein
monthly_revenue = 5000  # Monthly revenue ka estimate

# Estimate website value
total_value, monthly_value, yearly_value = estimate_value(domain_name, monthly_revenue)

# Create a DataFrame to display the results in a table format
results = {
    'Domain Name': [domain_name],
    'Monthly Revenue (₹)': [monthly_revenue],
    'Estimated Monthly Value (₹)': [monthly_value],
    'Estimated Yearly Value (₹)': [yearly_value],
    'Total Estimated Value (₹)': [total_value]
}

results_df = pd.DataFrame(results)

# Display the results
print(results_df)


     Domain Name  Monthly Revenue (₹)  Estimated Monthly Value (₹)  \
0  rosechair.com                 5000                       180100   

   Estimated Yearly Value (₹)  Total Estimated Value (₹)  
0                     2161200                     180100  


In [ ]:
!pip install python-whois pandas beautifulsoup4 openpyxl requests
import pandas as pd
import whois
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from urllib.parse import urlparse

# Function to extract emails and phone numbers from a given URL
def extract_emails_and_phones(url):
    try:
        web = requests.get(url)
        soup = BeautifulSoup(web.content, "html.parser")

        # Email regex
        email_pattern = r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'
        emails = re.findall(email_pattern, soup.get_text())
        cleaned_emails = [re.sub(r'[^a-zA-Z0-9._%+-@]', '', email) for email in emails]

        # Phone regex
        phone_pattern = r'(\+?\d{1,4}[\s-]?)?(\(?\d{2,4}\)?[\s-]?){0,1}(\d{3,4}[\s-]?\d{3,4}[\s-]?\d{0,4})'
        phone_numbers = [''.join(number).strip() for number in re.findall(phone_pattern, soup.get_text())]
        phone_numbers = list(set(phone_numbers))

        return cleaned_emails, phone_numbers
    except Exception as e:
        print(f"Error fetching data from {url}: {e}")
        return [], []

# Function to check for WordPress and its version
def check_wordpress(soup):
    is_wordpress = False
    wordpress_version = None

    if '/wp-content/' in str(soup) or '/wp-admin/' in str(soup):
        is_wordpress = True
        version_meta = soup.find('meta', {'name': 'generator'})
        if version_meta and 'WordPress' in version_meta.get('content', ''):
            match = re.search(r'WordPress (\d+\.\d+\.\d+)', version_meta.get('content'))
            if match:
                wordpress_version = match.group(1)
    return is_wordpress, wordpress_version

# Function to get website language
def get_website_language(soup):
    html_tag = soup.find('html')
    return html_tag.get('lang', 'Unknown') if html_tag else 'Unknown'

# 1. Domain Age ko calculate karna
def get_domain_age(domain_name):
    try:
        domain_info = whois.whois(domain_name)
        creation_date = domain_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        age = (datetime.now() - creation_date).days // 365  # Age in years
        return age
    except Exception as e:
        print(f"Error fetching domain age: {e}")
        return 0

# 2. Backlinks ko scrape karna
def get_unique_backlinks(domain_name):
    try:
        parsed_url = urlparse(domain_name)
        domain = parsed_url.netloc if parsed_url.netloc else parsed_url.path
        url = f"http://{domain}" if not parsed_url.scheme else domain_name
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        unique_backlinks = set([link.get('href') for link in links if link.get('href')])
        return len(unique_backlinks)
    except Exception as e:
        print(f"Error fetching backlinks for {domain_name}: {e}")
        return 0

# 3. Website Value ko estimate karna
def estimate_value(domain_name, monthly_revenue):
    # Domain Age
    domain_age = get_domain_age(domain_name)

    # Backlinks count
    backlinks_count = get_unique_backlinks(domain_name)

    # Simple valuation formula
    estimated_value = monthly_revenue * 24 + (domain_age * 5000) + (backlinks_count * 100)

    # Calculate monthly and yearly values
    monthly_value = estimated_value
    yearly_value = estimated_value * 12

    return estimated_value, monthly_value, yearly_value

# Function to fetch WHOIS data
def fetch_whois_data(domain_url):
    try:
        domain_info = whois.whois(domain_url)
        return {
            'domain_name': domain_info.domain_name,
            'registrar': domain_info.registrar,
            'creation_date': domain_info.creation_date,
            'expiration_date': domain_info.expiration_date
        }
    except Exception as e:
        print(f"WHOIS lookup failed for {domain_url}: {e}")
        return {
            'domain_name': 'Not available',
            'registrar': 'Not available',
            'creation_date': 'Not available',
            'expiration_date': 'Not available'
        }

# Load URLs from the Excel file
excel_file_path = 'urls.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file_path)
urls = df['URL'].dropna().tolist()

# List to store the results
results = []

# Iterate over each URL
for main_url in urls:
    print(f"Processing URL: {main_url}")

    try:
        web = requests.get(main_url)
        soup = BeautifulSoup(web.content, "html.parser")

        # Extract social media links
        all_links = [a['href'] for a in soup.find_all('a', href=True)]
        linkedin = next((link for link in all_links if "linkedin.com" in link), "Not found")
        instagram = next((link for link in all_links if "instagram.com" in link), "Not found")
        facebook = next((link for link in all_links if "facebook.com" in link), "Not found")
        twitter = next((link for link in all_links if "twitter.com" in link), "Not found")

        # Extract emails and phone numbers
        emails, phone_numbers = extract_emails_and_phones(main_url)

        # Check WordPress and version
        is_wordpress, wordpress_version = check_wordpress(soup)

        # Get website language
        website_language = get_website_language(soup)

        # Fetch WHOIS data
        domain_info = fetch_whois_data(main_url)

        # Estimate website value
        monthly_revenue = 5000  # Example, you can replace it with actual revenue
        estimated_value, monthly_value, yearly_value = estimate_value(main_url, monthly_revenue)

        # Add results to the list
        for email in emails or ["Not found"]:
            results.append({
                'website': main_url,
                'linkedin': linkedin,
                'instagram': instagram,
                'facebook': facebook,
                'twitter': twitter,
                'email': email,
                'number': ', '.join(phone_numbers) if phone_numbers else "Not found",
                'language': website_language,
                'is_wordpress': 'Yes' if is_wordpress else 'No',
                'wordpress_version': wordpress_version or 'Not available',
                'domain_name': domain_info['domain_name'],
                'registrar': domain_info['registrar'],
                'creation_date': domain_info['creation_date'],
                'expiration_date': domain_info['expiration_date'],
                'estimated_value': estimated_value,
                'estimated_monthly_value': monthly_value,
                'estimated_yearly_value': yearly_value
            })

    except Exception as e:
        print(f"Error processing {main_url}: {e}")
        results.append({
            'website': main_url,
            'linkedin': "Error processing",
            'instagram': "Error processing",
            'facebook': "Error processing",
            'twitter': "Error processing",
            'email': "Error processing",
            'number': "Error processing",
            'language': "Error processing",
            'is_wordpress': "Error processing",
            'wordpress_version': "Error processing",
            'domain_name': "Error processing",
            'registrar': "Error processing",
            'creation_date': "Error processing",
            'expiration_date': "Error processing",
            'estimated_value': "Error processing",
            'estimated_monthly_value': "Error processing",
            'estimated_yearly_value': "Error processing"
        })

# Convert results into a DataFrame
results_df = pd.DataFrame(results)
print("\nResults Table:")
print(results_df)

# Save the results to a new Excel file
output_excel_path = 'scraped_data_with_whois_and_value.xlsx'
results_df.to_excel(output_excel_path, index=False)
print(f"\nResults saved to {output_excel_path}")


Processing URL: https://superchargeyourmarketing.uk
Processing URL: https://sunrisetonoon.com
Processing URL: https://sunlifecreative.com
Processing URL: https://summitcreativeco.com
Processing URL: https://sublimemediagroup.com
Processing URL: https://styleadvertising.com
Processing URL: https://stuthecopywriter.co.uk
Processing URL: https://studiozmarketing.com
Processing URL: https://stuartwesselby.com
Processing URL: https://structuremedia.me
Processing URL: https://stratusdigitalagency.com
Processing URL: https://stratoscreativemarketing.com
Processing URL: https://strategynewmedia.com
Processing URL: https://strategiellc.com
Processing URL: https://storytellermn.com
Processing URL: https://steinerwebdesign.com
Processing URL: https://steelmarketing.org
Error processing https://steelmarketing.org: HTTPSConnectionPool(host='steelmarketing.org', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d91546caad0>, '

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.